# 决策树
- C4.5算法

In [1]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
class TreeNode(object):
    # 根据标签序列计算信息熵
    def get_Information_entropy(self, labels_list, n_samples):
        """
        :labels_list, ndarray, 标签列表、数组
        :n_samples, int, 总的类别数
        计算信息熵
        """
        _, label_counts = np.unique(labels_list, return_counts=True)
        p = label_counts * 1.0 / n_samples
        return -np.sum(p * np.log2(p))
    # 根据标签类别数计算信息熵
    def get_Information_entropy_with_counts(self, labels_count, n_samples):
        """
        :labels_count, ndarray, 标签数量列表、数组
        :n_samples, int, 总的标签数
        计算信息熵
        """
        p = labels_count * 1.0 / n_samples
        return -np.sum(p * np.log2(p))

    def __init__(self, data_x, data_y, segmentation_attr, attr_is_dispersed):
        '''
        :data_x: ndarray, 标签化的数据集x
        :data_y: ndarray, 标签化的数据集y
        :segmentation_attr, list, 当前可用的分割属性下标列表，例如[1,5,7],表示对于当前节点只有1、5、7可以
        被用作属性分割
        :attr_is_dispersed, ndarray, 属性是否为离散的，例如[1,0,1,0,1],则表示下标0、2、4的属性是离散的，
        '''
        self.data_x = data_x
        self.data_y = data_y
        self.segmentation_attr = segmentation_attr
        self.next_nodes = {}  # 存储子节点
        self.is_leaf = None
        self.seg_attr_index = -1  # 在该节点选择的分割属性
        self.seg_attr_value = -1  # 分割属性值，如果是离散的，则返回属性唯一值序列，否则返回只包含一
        # 个分割点属性值的序列

        n_samples, n_features = data_x.shape
        # 当前节点数据数不大于1、无可用分割属性、数据标签y全部一致的情况下，认定为子节点
        uniques_y = np.unique(self.data_y, return_counts=False)
        if n_samples <= 1 or len(self.segmentation_attr) == 0 or len(uniques_y) == 1:
            self.is_leaf = True
        else:  # 非叶子节点
            gain = self.get_Information_entropy(self.data_y, n_samples)  # 熵
            self.is_leaf = False
            # 根据可用分割属性segmentation_attr，以及属性离散/连续记录表attr_is_dispersed来确定最优
            # 分割属性,默认采用信息增益的方式，ID3算法
            temp_gain = -1  # 保存最小的条件熵
            temp_attr_index = -1  # 保存按哪个属性分割可以得到最小的条件熵

            # 保存该属性分割点的值，如果是离散的，则返回的列表包括属性的唯一值序列，否则仅
            # 返回包含分割点的属性值
            temp_attr_seg = None
            # 如果优属性是连续的，则返回升序属性序列的分割点下标，方便后续计算
            temp_continus_attr_seg_index = -1
            for attr_index in self.segmentation_attr:
                if attr_is_dispersed[attr_index] == 1:  # 离散值
                    uniques_attr, uniques_attr_counts = np.unique(
                        self.data_x[:, attr_index], return_counts=True)
                    temp_information_gains = []
                    for cur_attr_label in uniques_attr:
                        cur_mask = (self.data_x[:, attr_index] == cur_attr_label)
                        temp_information_gains.append(
                            self.get_Information_entropy(self.data_y[cur_mask], len(cur_mask)))
                    cur_gain = np.sum((uniques_attr_counts * 1.0 / n_samples) *
                                      temp_information_gains)
                    
                    gain_cur_attr = self.get_Information_entropy_with_counts(
                        uniques_attr_counts, n_samples)
                    if temp_gain < (gain - cur_gain) / gain_cur_attr:
                        temp_gain = (gain - cur_gain) / gain_cur_attr
                        temp_attr_index = attr_index
                        temp_attr_seg = uniques_attr    
                    
                else:  # 连续值,需要寻找一个最优的二分点来分割数据，需要进行n_samples-1次尝试，
                    # 对于ID3算法，最优二分点处条件熵最小即可，对于C4.5，则需要计算归一化的信息
                    # 增益，来决定连续属性的最优二分割点
                    sort_index = np.argsort(self.data_x[:, attr_index])
                    temp_continus_attr_seg = -1
                    temp_continus_gain = -1
                    temp_sort_index = -1
                    for i in range(n_samples - 1):
                        temp_continus_seg_value = (data_x[i, attr_index] + \
                                                   data_x[i + 1, attr_index]) / 2.0
                        temp_continuous_left = self.get_Information_entropy(
                            data_y[sort_index[:i + 1]], i + 1)
                        temp_continuous_right = self.get_Information_entropy(
                            data_y[sort_index[i + 1:]], n_samples - i - 1)
                        # 计算连续属性的条件熵
                        cur_continus_gains = (i + 1.0) / n_samples * temp_continuous_left + \
                                             (n_samples - i - 1.0) / n_samples * temp_continuous_right
                        
                        # 计算当前二分点处的固有值， 在不同的二分割点处，该属性具有不同的“固有值”
                        cur_seg_iv = self.get_Information_entropy_with_counts(
                        np.array([i+1,n_samples - i - 1]), n_samples)
                        if temp_continus_gain < (gain - cur_continus_gains)/cur_seg_iv:
                            temp_continus_gain = (gain - cur_continus_gains)/cur_seg_iv
                            temp_continus_attr_seg = temp_continus_seg_value
                            temp_sort_index = i
                            
                    cur_gain = temp_continus_gain
                    if temp_gain < cur_gain:
                        temp_gain = cur_gain
                        temp_attr_index = attr_index
                        temp_attr_seg = np.array([temp_continus_attr_seg])
                        temp_continus_attr_seg_index = temp_sort_index
                            
            # 利用最优属性进行划分，并创造该节点的子节点，保存在self.next_nodes结构中

            self.seg_attr_index = temp_attr_index  # 该节点的分割属性（轴）的下标
            self.seg_attr_value = temp_attr_seg

            # 最优属性是离散值
            if attr_is_dispersed[temp_attr_index] == 1:
                self.segmentation_attr.remove(temp_attr_index)  # 从备用分割属性列表中删除最优属性
                for cur_attr_label in temp_attr_seg:
                    cur_mask = (self.data_x[:, self.seg_attr_index] == cur_attr_label)
                    self.next_nodes[cur_attr_label] = TreeNode(self.data_x[cur_mask],
                                                               self.data_y[cur_mask],
                                                               self.segmentation_attr,
                                                               attr_is_dispersed)
            else:  # 最优属性是连续值
                sort_index = np.argsort(self.data_x[:, temp_attr_index])
                # 不大于分割属性的子节点
                self.next_nodes[0] = TreeNode(
                    data_x[sort_index[:temp_continus_attr_seg_index + 1]],
                    data_y[sort_index[:temp_continus_attr_seg_index + 1]],
                    self.segmentation_attr,
                    attr_is_dispersed)  # left
                # 大于分割属性的子节点
                self.next_nodes[1] = TreeNode(
                    data_x[sort_index[temp_continus_attr_seg_index + 1:]],
                    data_y[sort_index[temp_continus_attr_seg_index + 1:]],
                    self.segmentation_attr,
                    attr_is_dispersed)  # right

In [18]:
class DecisionTree(object):
    def __init__(self, train_x, train_y, attributes_classs=None,
                 criterion='entropy'):
        self.train_x = train_x
        self.train_y = train_y

        # 确定属性是连续/离散的,当属性中唯一值数量多于N/2时，认定为连续值
        n_samples, n_features = self.train_x.shape
        if attributes_classs is None:
            attributes_classs = [0] * n_samples
            for i in range(n_features):
                uniques_i = np.unique(self.train_x[:, i])
                if uniques_i * 3 > n_samples:
                    attributes_classs[i] = 0
        self.attributes_classs = attributes_classs
        # 对离散属性、label做encoder处理
        self.xLabelEncoders = []
        for i in range(len(self.attributes_classs)):
            if self.attributes_classs[i] == 1:  # 离散属性
                cur_encoder = LabelEncoder()
                cur_encoder.fit(self.train_x[:, i])
                self.train_x[:, i] = cur_encoder.transform(self.train_x[:, i])
                self.xLabelEncoders.append(cur_encoder)
            else:
                self.xLabelEncoders.append(None)
        self.yLabelEncoders = LabelEncoder()
        self.yLabelEncoders.fit(train_y)
        self.train_y = self.yLabelEncoders.transform(self.train_y)
        self.root = None  # 根节点

    def train(self):
        self.root = TreeNode(self.train_x, self.train_y, range(len(self.attributes_classs)),
                             self.attributes_classs)

    def fit(self):
        self.train()

    def predict(self, test_x):
        # 从根节点开始遍历树形结构，
        if self.root is None:
            raise RuntimeError("value is None, error")
        # 首先将测试数据集的离散属性LAbelencoder,然后进行预测
        for (i, x_label_encoder) in enumerate(self.xLabelEncoders):
            if x_label_encoder is not None:
                test_x[:, i] = x_label_encoder.transform(test_x[:, i],)
        n_samples, _ = test_x.shape
        pre = np.zeros(n_samples)
        for i in xrange(n_samples):
            cur_node = self.root
            while cur_node.is_leaf is False:
                cur_attr_index = cur_node.seg_attr_index  # 分割属性下标
                if self.attributes_classs[cur_attr_index] == 1:  # 离散属性
                    if test_x[i][cur_attr_index] in cur_node.next_nodes.keys():
                        cur_node = cur_node.next_nodes[test_x[i][cur_attr_index]]
                        break
                    else:
                        raise Exception('the attr is not in Decision Tree')
                else:  # 连续属性
                    if test_x[i][cur_attr_index] <= cur_node.seg_attr_value:
                        cur_node = cur_node.next_nodes[0]
                    else:
                        cur_node = cur_node.next_nodes[1]
            # 找到了叶子节点，可以展开预测动作了，基本的投票原则
            pre[i] = np.argmax(np.bincount(cur_node.data_y))
        pre = pre.astype(np.int)
        return self.yLabelEncoders.inverse_transform(pre)  # 反向转换

## 训练决策树

In [12]:
def load_data(path = '../data/heart.csv'):
    weather = pd.read_csv(path,sep=',')
    data = weather.values
    data_x = data[:,:-1]
    data_y = data[:, -1]
    return data_x, data_y

In [13]:
data_x, data_y = load_data()

In [30]:
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.2)

In [31]:
dt = DecisionTree(train_x, train_y, [0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1])

In [32]:
dt.train()

/home/hao/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in double_scalars


In [33]:
pre_y = dt.predict(test_x)

In [34]:
np.sum(pre_y == test_y)*1.0 / test_y.shape[0]

0.65573770491803274